In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [57]:
df = pd.read_csv('ionosphere.csv')
df['34'] = df['34'].map({'g': 1, 'b': 0})
print(df.head())

   0  1        2        3        4        5        6        7        8  \
0  1  0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1  1  0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2  1  0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3  1  0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4  1  0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   

         9  ...       25       26       27       28       29       30  \
0  0.03760  ... -0.51171  0.41078 -0.46168  0.21266 -0.34090  0.42267   
1 -0.04549  ... -0.26569 -0.20468 -0.18401 -0.19040 -0.11593 -0.16626   
2  0.01198  ... -0.40220  0.58984 -0.22145  0.43100 -0.17365  0.60436   
3  0.00000  ...  0.90695  0.51613  1.00000  1.00000 -0.20099  0.25682   
4 -0.16399  ... -0.65158  0.13290 -0.53206  0.02431 -0.62197 -0.05707   

        31       32       33  34  
0 -0.54487  0.18641 -0.45300   1  
1 -0.06288 -0.13738 -0.02447   0  
2 -0.24180 

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

class CSV_Dataset(Dataset):
    def __init__(self, csv_file, path = None, transform=None):
        if path is None:
            self.data = pd.read_csv(csv_file)
        else:
            self.data = pd.read_csv(path + csv_file)        
        self.transform = transform
        
        self.x = torch.FloatTensor(self.data.iloc[:, :-1].values)
        self.y = torch.LongTensor(LabelEncoder().fit_transform(self.data.iloc[:, -1].values))
    
    def _split(self, test_size):
        x_train, x_test, y_train, y_test = train_test_split(self.x, self.y, test_size=test_size)
        self.x = x_train
        self.y = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        self._split(.2)
        return self.x[item], self.y[item]

In [37]:
class Net(nn.Module):
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_inputs, n_hidden)
        self.act1 = nn.ReLU()
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(n_hidden, n_outputs)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        x = self.output(x)
        return x

In [38]:
def train(model, train_loader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [39]:
from sklearn.metrics import accuracy_score

def eval(model, test_loader):
    y_pred = []
    y_true = []
    for x, y in test_loader:
        y_hat = model(x)
        y_hat = torch.argmax(y_hat, dim=1)
        y_pred.append(y_hat)
        y_true.append(y)
    y_pred = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    return accuracy_score(y_true, y_pred)

In [41]:
dataset = CSV_Dataset('ionosphere.csv')


In [42]:
dataset._split(.2)

In [44]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [45]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def predict(model, x):
    y_hat = model(x)
    y_hat = torch.argmax(y_hat, dim=1)
    return y_hat

In [47]:
model = Net(34, 100, 2)

In [48]:
train(model, train_loader, optim.Adam(model.parameters(), lr=0.001), nn.BCELoss(), 100)

IndexError: index 272 is out of bounds for dimension 0 with size 224

In [ ]:
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [ ]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv('ionosphere.csv')
        print(df.head())
        df['34'] = df['34'].map({'g': 1, 'b': 0})
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [ ]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [ ]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [ ]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [ ]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

In [ ]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [ ]:
# prepare the data
path = './ionosphere.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(34)
# train the model
train_model(train_dl, model)
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))